# Merge CoW and prep for time series integration

In [1]:
import pandas as pd
import numpy as np

read in the relevant tables. don't need the war transitions table

In [2]:
cow_par = pd.read_csv("../Data/CoW/Wrangled/war_participants.csv")
cow_pol = pd.read_csv("../Data/CoW/Wrangled/polities.csv")
cow_war = pd.read_csv("../Data/CoW/Wrangled/wars.csv", usecols=['WarID', 'WarTypeName', 'IsIntervention', 'IsInternational'])
cow_loc = pd.read_csv("../Data/CoW/Wrangled/war_locations.csv")

limit the cow_par table to only those rows where the participant is a state

In [3]:
cow_pol_states = cow_pol[cow_pol['PolityType']=='State']
cow_pol_states_list = list(cow_pol_states['PolityID'].unique())

cow_par = cow_par[cow_par['PolityID'].isin(cow_pol_states_list)]

expand the start and end dates so there is one row per year for each war-participant observation, then reduce to only rows with year of 1946 or greater

In [4]:
cow_par['EndDate'] = cow_par['EndDate'].fillna('2008-12-31')
cow_par['StartDate'] = pd.to_datetime(cow_par['StartDate'])
cow_par['EndDate'] = pd.to_datetime(cow_par['EndDate'])

cow_par['StartDateClean'] = cow_par['StartDate'].apply(lambda dt: dt.replace(day=1, month=1))
cow_par['EndDateClean'] = cow_par['EndDate'].apply(lambda dt: dt.replace(day=1, month=1))


cow_par_ts = pd.concat([pd.DataFrame({'year': pd.date_range(row.StartDateClean, row.EndDateClean, freq='YS'),
                                      'StartDate': row.StartDate,
                                      'EndDate': row.EndDate,
                                      'cow_id': row.PolityID, 
                                      'war_id': row.WarID, 
                                      'IsInitiator': row.IsInitiator, 
                                      'Outcome': row.Outcome, 
                                      'total_deaths': row.Deaths}, 
                                 columns=['year', 'StartDate', 'EndDate', 'cow_id', 'war_id', 'IsInitiator', 'Outcome', 'total_deaths']) 
                                 for i, row in cow_par.iterrows()], ignore_index=True)
cow_par_ts['year'] = cow_par_ts['year'].dt.year
cow_par_ts = cow_par_ts[cow_par_ts['year'] > 1945].reset_index(drop=True)
cow_par_ts

,year,StartDate,EndDate,cow_id,war_id,IsInitiator,Outcome,total_deaths
0,1947,1947-10-26,1949-01-01,750,147,1,6,2500.0
1,1948,1947-10-26,1949-01-01,750,147,1,6,2500.0
2,1949,1947-10-26,1949-01-01,750,147,1,6,2500.0
3,1947,1947-10-26,1949-01-01,770,147,0,6,1000.0
4,1948,1947-10-26,1949-01-01,770,147,0,6,1000.0
...,...,...,...,...,...,...,...,...
1528,1949,1948-04-03,1949-05-01,732,1573,0,1,NaN
1529,1968,1968-09-01,1971-10-06,698,1577,0,6,NaN
1530,1969,1968-09-01,1971-10-06,698,1577,0,6,NaN
1531,1970,1968-09-01,1971-10-06,698,1577,0,6,NaN


find the number of days spend in the conflict for the given year

In [5]:
def days_in_conflict(row):
    startyear = row.StartDate.year
    endyear = row.EndDate.year
    year = row.year
    daysinyear = 365
    if (year%4==0) & (year%100==0):
        daysinyear = 366
    
    lastday = pd.Timestamp(year=year, month=12, day=31)
    firstday = pd.Timestamp(year=year, month=1, day=1)
    
    days = 0
    
    if (startyear == year) & (endyear == year):
        days = (row.EndDate - row.StartDate).days + 1
    elif (startyear == year) & (endyear != year):
        days = (lastday - row.StartDate).days + 1
    elif (startyear != year) & (endyear == year):
        days = (row.EndDate - firstday).days + 1
    else:
        days = daysinyear
    
    return (days / daysinyear)

In [6]:
cow_par_ts['TimeAtWar'] = cow_par_ts.apply(days_in_conflict, axis=1).round(3)
cow_par_ts

,year,StartDate,EndDate,cow_id,war_id,IsInitiator,Outcome,total_deaths,TimeAtWar
0,1947,1947-10-26,1949-01-01,750,147,1,6,2500.0,0.184
1,1948,1947-10-26,1949-01-01,750,147,1,6,2500.0,1.000
2,1949,1947-10-26,1949-01-01,750,147,1,6,2500.0,0.003
3,1947,1947-10-26,1949-01-01,770,147,0,6,1000.0,0.184
4,1948,1947-10-26,1949-01-01,770,147,0,6,1000.0,1.000
...,...,...,...,...,...,...,...,...,...
1528,1949,1948-04-03,1949-05-01,732,1573,0,1,NaN,0.332
1529,1968,1968-09-01,1971-10-06,698,1577,0,6,NaN,0.334
1530,1969,1968-09-01,1971-10-06,698,1577,0,6,NaN,1.000
1531,1970,1968-09-01,1971-10-06,698,1577,0,6,NaN,1.000


determine if conflict was fought in same region where participant is

In [8]:
wars = list(cow_par_ts['war_id'].unique())
cow_loc = cow_loc[cow_loc['WarID'].isin(wars)]
cow_loc2 = cow_loc.groupby('WarID').agg({'Region':lambda x: ','.join(list(x))})

In [9]:
cow_warloc = cow_war.merge(cow_loc2, on='WarID')
cow_warloc = cow_warloc.rename(columns={'WarID': 'war_id', 'WarTypeName': 'war_type', 'Region': 'war_region'})
cow_merged = cow_par_ts.merge(cow_warloc, on=['war_id'], how='left')

,year,StartDate,EndDate,cow_id,war_id,IsInitiator,Outcome,total_deaths,TimeAtWar,war_type,IsIntervention,IsInternational,war_region
0,1947,1947-10-26,1949-01-01,750,147,1,6,2500.0,0.184,Inter-State War,NaN,NaN,Asia
1,1948,1947-10-26,1949-01-01,750,147,1,6,2500.0,1.000,Inter-State War,NaN,NaN,Asia
2,1949,1947-10-26,1949-01-01,750,147,1,6,2500.0,0.003,Inter-State War,NaN,NaN,Asia
3,1947,1947-10-26,1949-01-01,770,147,0,6,1000.0,0.184,Inter-State War,NaN,NaN,Asia
4,1948,1947-10-26,1949-01-01,770,147,0,6,1000.0,1.000,Inter-State War,NaN,NaN,Asia
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1528,1949,1948-04-03,1949-05-01,732,1573,0,1,NaN,0.332,Non-State War,NaN,NaN,Asia
1529,1968,1968-09-01,1971-10-06,698,1577,0,6,NaN,0.334,Non-State War,NaN,NaN,Middle East
1530,1969,1968-09-01,1971-10-06,698,1577,0,6,NaN,1.000,Non-State War,NaN,NaN,Middle East
1531,1970,1968-09-01,1971-10-06,698,1577,0,6,NaN,1.000,Non-State War,NaN,NaN,Middle East


In [10]:
def get_state_region(row):
    state = row.cow_id
    region = "none"
    
    if state < 200:
        region = "W. Hemisphere"
    elif state < 400:
        region = "Europe"
    elif state < 600:
        region = "Africa"
    elif state < 700:
        region = "Middle East"
    elif state < 900:
        region = "Asia"
    else:
        region = "Oceania"
    
    return region

In [25]:
cow_merged['state_region'] = cow_merged.apply(get_state_region, axis=1)
cow_merged['IsSameRegion'] = cow_merged.apply(lambda x: x.state_region in x.war_region.split(','), axis=1).astype(int)
cow_merged

,year,StartDate,EndDate,cow_id,war_id,IsInitiator,Outcome,total_deaths,TimeAtWar,war_type,IsIntervention,IsInternational,war_region,state_region,IsSameRegion
0,1947,1947-10-26,1949-01-01,750,147,1,6,2500.0,0.184,Inter-State War,NaN,NaN,Asia,Asia,1
1,1948,1947-10-26,1949-01-01,750,147,1,6,2500.0,1.000,Inter-State War,NaN,NaN,Asia,Asia,1
2,1949,1947-10-26,1949-01-01,750,147,1,6,2500.0,0.003,Inter-State War,NaN,NaN,Asia,Asia,1
3,1947,1947-10-26,1949-01-01,770,147,0,6,1000.0,0.184,Inter-State War,NaN,NaN,Asia,Asia,1
4,1948,1947-10-26,1949-01-01,770,147,0,6,1000.0,1.000,Inter-State War,NaN,NaN,Asia,Asia,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1528,1949,1948-04-03,1949-05-01,732,1573,0,1,NaN,0.332,Non-State War,NaN,NaN,Asia,Asia,1
1529,1968,1968-09-01,1971-10-06,698,1577,0,6,NaN,0.334,Non-State War,NaN,NaN,Middle East,Middle East,1
1530,1969,1968-09-01,1971-10-06,698,1577,0,6,NaN,1.000,Non-State War,NaN,NaN,Middle East,Middle East,1
1531,1970,1968-09-01,1971-10-06,698,1577,0,6,NaN,1.000,Non-State War,NaN,NaN,Middle East,Middle East,1


find how many years each conflict lasted for so the total_deaths can be normalized to the average per year.

In [32]:
cow_merged_wars = cow_merged.groupby(['war_id', 'cow_id']).agg({'year': 'count'})
cow_merged_wars = cow_merged_wars.reset_index().rename(columns={'year':'num_years'})

cow_merged2 = cow_merged.merge(cow_merged_wars, on=['war_id', 'cow_id'])
cow_merged2['avg_deaths'] = (cow_merged2['total_deaths'] / cow_merged2['num_years'])
cow_merged2['avg_deaths'] = cow_merged2['avg_deaths'].round(0).astype('Int64')
cow_merged2

,year,StartDate,EndDate,cow_id,war_id,IsInitiator,Outcome,total_deaths,TimeAtWar,war_type,IsIntervention,IsInternational,war_region,state_region,IsSameRegion,num_years,avg_deaths
0,1947,1947-10-26,1949-01-01,750,147,1,6,2500.0,0.184,Inter-State War,NaN,NaN,Asia,Asia,1,3,833
1,1948,1947-10-26,1949-01-01,750,147,1,6,2500.0,1.000,Inter-State War,NaN,NaN,Asia,Asia,1,3,833
2,1949,1947-10-26,1949-01-01,750,147,1,6,2500.0,0.003,Inter-State War,NaN,NaN,Asia,Asia,1,3,833
3,1947,1947-10-26,1949-01-01,770,147,0,6,1000.0,0.184,Inter-State War,NaN,NaN,Asia,Asia,1,3,333
4,1948,1947-10-26,1949-01-01,770,147,0,6,1000.0,1.000,Inter-State War,NaN,NaN,Asia,Asia,1,3,333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1528,1949,1948-04-03,1949-05-01,732,1573,0,1,NaN,0.332,Non-State War,NaN,NaN,Asia,Asia,1,2,NaN
1529,1968,1968-09-01,1971-10-06,698,1577,0,6,NaN,0.334,Non-State War,NaN,NaN,Middle East,Middle East,1,4,NaN
1530,1969,1968-09-01,1971-10-06,698,1577,0,6,NaN,1.000,Non-State War,NaN,NaN,Middle East,Middle East,1,4,NaN
1531,1970,1968-09-01,1971-10-06,698,1577,0,6,NaN,1.000,Non-State War,NaN,NaN,Middle East,Middle East,1,4,NaN


create dummy variables for each war type

In [33]:
cow_merged2['war_type'].unique()

array(['Inter-State War ', 'Extra-State War ', 'Intra-State War ',
       'Non-State War '], dtype=object)

In [34]:
type_map = {'Inter-State War':'InterState', 'Extra-State War':'ExtraState', 'Intra-State War':'IntraState', 'Non-State War':'NonState'}
cow_merged2['war_type'] = cow_merged2['war_type'].map(type_map)
type_dummy = pd.get_dummies(cow_merged2['war_type'], prefix='type')
cow_merged2 = pd.concat([cow_merged2, type_dummy], axis=1)

create dummy variables for each war outcome type

note - should this be limited to only the last year of the war?

In [35]:
cow_merged2['Outcome'].unique()

array([6, 1, 2, 4, 3, 7, 5])

In [36]:
outcome_map = {1: 'win', 2: 'lose', 3: 'tied', 4: 'war-transitioned', 5: 'ongoing2008', 6: 'stalemate', 7: 'cont-conflict'}
cow_merged2['Outcome'] = cow_merged2['Outcome'].map(outcome_map)
outcome_dummy = pd.get_dummies(cow_merged2['Outcome'], prefix='outcome')
cow_merged2 = pd.concat([cow_merged2, outcome_dummy], axis=1)

group by country and year (so wars get crunched together) and compute aggregates - count number of unique wars, sum the dummy variables, and sum the average deaths.

In [37]:
cow_merged2.columns

Index(['year', 'StartDate', 'EndDate', 'cow_id', 'war_id', 'IsInitiator',
       'Outcome', 'total_deaths', 'TimeAtWar', 'war_type', 'IsIntervention',
       'IsInternational', 'war_region', 'state_region', 'IsSameRegion',
       'num_years', 'avg_deaths', 'type_ExtraState', 'type_InterState',
       'type_IntraState', 'type_NonState', 'outcome_cont-conflict',
       'outcome_lose', 'outcome_ongoing2008', 'outcome_stalemate',
       'outcome_tied', 'outcome_war-transitioned', 'outcome_win'],
      dtype='object')

In [41]:
cow_gb = cow_merged2.groupby(['cow_id', 'year']).agg({'war_id': 'nunique', 
                                                      'IsInitiator': 'sum', 
                                                      'avg_deaths': 'sum',
                                                      'IsSameRegion': 'sum',
                                                      'type_ExtraState': 'sum', 
                                                      'type_InterState': 'sum', 
                                                      'type_IntraState': 'sum',
                                                      'type_NonState': 'sum', 
                                                      'outcome_cont-conflict': 'sum', 
                                                      'outcome_lose': 'sum',
                                                      'outcome_ongoing2008': 'sum', 
                                                      'outcome_stalemate': 'sum', 
                                                      'outcome_tied': 'sum',
                                                      'outcome_war-transitioned': 'sum', 
                                                      'outcome_win': 'sum'})
cow_gb = cow_gb.rename(columns={'war_id': 'war_count'})
cow_gb = cow_gb.add_prefix('cow_')
cow_gb

cow_war_count  cow_IsInitiator  cow_avg_deaths  cow_IsSameRegion  \
cow_id year                                                                     
2      1950              1                0           13622                 0   
       1951              1                0           13622                 0   
       1952              1                0           13622                 0   
       1953              1                0           13622                 0   
       1958              1                0               1                 0   
...                    ...              ...             ...               ...   
900    2008              2                2               1                 0   
910    1989              1                1               0                 0   
       1990              1                1               0                 0   
       1991              1                1               0                 0   
       1992              1                1               0                 0   

             cow_type_ExtraState  cow_type_InterState  cow_type_IntraState  \
cow_id year                                                                  
2      1950                    0                    1                    0   
       1951                    0                    1                    0   
       1952                    0                    1                    0   
       1953                    0                    1                    0   
       1958                    0                    0                    1   
...                          ...                  ...                  ...   
900    2008                    2                    0                    0   
910    1989                    0                    0                    1   
       1990                    0                    0                    1   
       1991                    0                    0                    1   
       1992                    0                    0                    1   

             cow_type_NonState  cow_outcome_cont-conflict  cow_outcome_lose  \
cow_id year                                                                   
2      1950                  0                          0                 0   
       1951                  0                          0                 0   
       1952                  0                          0                 0   
       1953                  0                          0                 0   
       1958                  0                          0                 0   
...                        ...                        ...               ...   
900    2008                  0                          0                 0   
910    1989                  0                          1                 0   
       1990                  0                          1                 0   
       1991                  0                          1                 0   
       1992                  0                          1                 0   

             cow_outcome_ongoing2008  cow_outcome_stalemate  cow_outcome_tied  \
cow_id year                                                                     
2      1950                        0                      1                 0   
       1951                        0                      1                 0   
       1952                        0                      1                 0   
       1953                        0                      1                 0   
       1958                        0                      0                 0   
...                              ...                    ...               ...   
900    2008                        2                      0                 0   
910    1989                        0                      0                 0   
       1990                        0                      0                 0   
       1991              

merge onto base and fill NAs with 0

In [46]:
base_ts = pd.read_csv("../Data/FINAL/countrycodes_ts-base.csv", usecols=['country_name', 'year', 'cow_id']).dropna()
cow_ts = pd.read_csv("../Data/CoW/Raw/system2016.csv", usecols=['ccode', 'year'])

In [51]:
cow_ts = cow_ts.rename(columns={'ccode':'cow_id'})
cow_ts = cow_ts[cow_ts['year']>1945]
cow_ts['inCoW'] = 1
merge_ts = base_ts.merge(cow_ts, on=['cow_id', 'year'], how='outer')
merge_ts

,country_name,year,cow_id,inCoW
0,Afghanistan,1946,700.0,1.0
1,Afghanistan,1947,700.0,1.0
2,Afghanistan,1948,700.0,1.0
3,Afghanistan,1949,700.0,1.0
4,Afghanistan,1950,700.0,1.0
...,...,...,...,...
10905,NaN,1972,816.0,1.0
10906,NaN,1973,816.0,1.0
10907,NaN,1974,816.0,1.0
10908,NaN,1975,816.0,1.0


In [52]:
merge_ts[merge_ts['country_name'].isna()]

,country_name,year,cow_id,inCoW
10887,NaN,1954,816.0,1.0
10888,NaN,1955,816.0,1.0
10889,NaN,1956,816.0,1.0
10890,NaN,1957,816.0,1.0
10891,NaN,1958,816.0,1.0
10892,NaN,1959,816.0,1.0
10893,NaN,1960,816.0,1.0
10894,NaN,1961,816.0,1.0
10895,NaN,1962,816.0,1.0
10896,NaN,1963,816.0,1.0


In [ ]:
cow_gb.to_csv("../Data/FINAL/cow.csv")